In [1]:
import os
import sys
from pathlib import Path



In [2]:
%pwd

'd:\\Time Series Forecasting\\Time-Series-Forecasting-with-XGBoost\\notebooks'

In [3]:
# Set up project paths
PROJECT_ROOT = Path.cwd().parents[0]
data_path = PROJECT_ROOT / 'data' / 'raw'

In [ ]:
# Data validation 

